In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 
src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset_utils import create_nuscenes_odometry_dataset
from dataset.filters.filter_list import FilterList
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose
import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_unite_gt, remove_semantics, merge_chunks_unite_instances2
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil
from tqdm import tqdm
from scipy.spatial import KDTree
import torch.nn as nn

from utils.UIS.utils import *
from utils.UIS.minkunet import *
from utils.UIS.collations import *
from utils.UIS.corr_utils import *
from utils.UIS.pcd_preprocess import *

In [ ]:
import sklearn
from sklearn.cluster import DBSCAN, HDBSCAN
import hdbscan
import yaml 
import scipy 

config = 'utils/UIS/instance_seg.yaml'
cfg = yaml.safe_load(open(config))
params = cfg

set_deterministic()
model = MinkUNet(in_channels=4, out_channels=96).type(torch.FloatTensor)
#checkpoint = torch.load(cfg['model']['checkpoint'], map_location=torch.device('cuda'))
checkpoint = torch.load('utils/UIS/epoch199_model_segcontrast.pt', map_location=torch.device('cuda'))
model.cuda()
#model.load_state_dict(checkpoint[cfg['model']['checkpoint_key']])
model.load_state_dict(checkpoint[cfg['model']['checkpoint_key']])
model.dropout = nn.Identity()


for param in model.parameters():
        param.require_grads = False


def uniform_down_sample_with_indices(points, every_k_points):
        # Create a new point cloud for the downsampled output

        # List to hold the indices of the points that are kept
        indices = []

        # Iterate over the points and keep every k-th point
        for i in range(0, points.shape[0], every_k_points):
            indices.append(i)

        return indices

def downsample_chunk(points):
        num_points_to_sample = 30000
        every_k_points = int(
            points.shape[0] /
            num_points_to_sample)
        indeces = uniform_down_sample_with_indices(
            points, every_k_points)


        return points[indeces]
        
def segcontrast_preprocessing(p,sem_labels,resolution=0.05,num_points='inf'):
    coord_p, feats_p, cluster_p = point_set_to_coord_feats(p, sem_labels, resolution, num_points)
    return coord_p, feats_p, cluster_p


def UIS3D_clustering(pcd_nonground_chunk, pcd_ground_chunk,center_id,center_position,
                        eps=0.3, min_samples=10,tarl=False):
    """
    Perform DBSCAN clustering on the point cloud data.

    :param cur_pcd: Current point cloud for clustering.
    :param pcd_all: All point cloud data.
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    :return: Cluster labels for each point in the point cloud.
    """
    
    inliers = get_statistical_inlier_indices(pcd_ground_chunk)
    ground_inliers = get_subpcd(pcd_ground_chunk, inliers)
    mean_hight = np.mean(np.asarray(ground_inliers.points)[:,2])
    in_idcs = np.where(np.asarray(ground_inliers.points)[:,2] < (mean_hight + 0.2))[0]
    cut_hight = get_subpcd(ground_inliers, in_idcs)
    cut_hight.paint_uniform_color([0, 0, 0])
    
    in_idcs = None
    
    #in_idcs = np.where(np.asarray(pcd_nonground_chunk.points)[:,2] > (mean_hight + 0.05))[0]
    #pcd_nonground_corrected = get_subpcd(pcd_nonground_chunk, in_idcs)
    pcd_nonground_corrected = pcd_nonground_chunk
    
    merge_orig = pcd_nonground_corrected + cut_hight
    
    pcd_nonground_downsampled = o3d.geometry.PointCloud()
    pts_downsampled = downsample_chunk(np.asarray(pcd_nonground_corrected.points))
    pcd_nonground_downsampled.points = o3d.utility.Vector3dVector(pts_downsampled)
    
    ground_downsampled = o3d.geometry.PointCloud()
    pts_downsampled_ground = downsample_chunk(np.asarray(cut_hight.points))
    ground_downsampled.points = o3d.utility.Vector3dVector(pts_downsampled_ground)
    
    
    
    #clustering = DBSCAN(eps=eps, min_samples=min_samples)
    #clustering = HDBSCAN(min_cluster_size=10).fit(pts_downsampled)
    clustering = hdbscan.HDBSCAN(algorithm='best', alpha=1., approx_min_span_tree=True,
                                gen_min_span_tree=True, leaf_size=100,
                                metric='euclidean', min_cluster_size=10, min_samples=None
                            )
    clustering.fit(pts_downsampled)
    
    
    merged_chunk = pcd_nonground_downsampled + ground_downsampled
    
    labels_nonground = clustering.labels_.reshape(-1,1) + 2
    points = np.asarray(merged_chunk.points)
    labels = np.ones((points.shape[0], 1)) * -1
    
    ground_labels = np.zeros(points.shape[0]) * -1
    non_ground_size = np.asarray(pcd_nonground_downsampled.points).shape[0]
    ground_labels[:non_ground_size] = 1
    labels[:non_ground_size] = labels_nonground
    pcd_cur = color_pcd_by_labels(merged_chunk,labels)
    #o3d.visualization.draw_geometries([pcd_cur])
    ins, num_pts = np.unique(labels, return_counts=True)
    
    mask = np.ones(labels.shape[0], dtype=bool)
    mask[non_ground_size:] = False
    points = np.concatenate((points,np.ones((points.shape[0],1))),1)
    mean_x = points[:,0].mean() 
    mean_y = points[:,1].mean() 
    mean_z = points[:,2].mean() 
    
    points[:,0] -= mean_x
    points[:,1] -= mean_y
    points[:,2] -= mean_z
    
    
    
    ###local features from tarl : note can not be used as network saliency is needed 
 
    coord_p,feats_p,cluster_p = segcontrast_preprocessing(points,labels)
    
    slc_full = np.zeros((points.shape[0],), dtype=int)
    pred_ins_full = np.zeros((points.shape[0],), dtype=int)
    for cluster in ins: 
            cls_points = np.where(cluster_p == cluster)[0]
            
            if cluster == 0 or len(cls_points) <= 100:
                continue
            # get cluster
            cluster_center = coord_p[cls_points].mean(axis=0)

            # crop a ROI around the cluster
            window_points = crop_region(coord_p,cluster_p, cluster, 20)

            # skip when ROI is empty        
            if not np.sum(window_points):
                continue

            # get closest point to the center
            center_dists = np.sqrt(np.sum((coord_p[window_points] - cluster_center)**2, axis=-1))
            cluster_center = np.argmin(center_dists)

            # build input only with the ROI points
            x_forward = numpy_to_sparse_tensor(coord_p[window_points][np.newaxis, :, :], feats_p[window_points][np.newaxis, :, :])
            
            # forward pass ROI 
            model.eval()
            x_forward.F.requires_grad = True
            out = model(x_forward.sparse())
            out = out.slice(x_forward)

            # reset grads to compute saliency
            x_forward.F.grad = None

            # compute saliency for the point in the center
            slc = get_cluster_saliency(x_forward, out, np.where(cluster_p[window_points] == cluster)[0])
            slc_ = slc.copy()

            # place the computed saliency into the full point cloud for comparison
            slc_full[window_points] = np.maximum(slc_full[window_points], slc)

            # build graph representation
            G = build_graph(out.F.detach().cpu().numpy(),
                            slc[:,np.newaxis],
                            coord_p[window_points],
                            cluster_center,
                            np.sum(cluster_p == cluster),
                            params,
                            ground_labels[window_points],
                            np.where(cluster_p[window_points] == cluster)[0],
                        )
            # perform graph cut
            #G = scipy.sparse.csr_matrix(G) -> try out this line 
            ins_points = graph_cut(G)
            # create point-wise prediction matrix
            pred_ins = np.zeros((len(x_forward),)).astype(int)
            if len(ins_points) != 0:
                pred_ins[ins_points] = cluster
            
            # ignore assigned ground labels
            ins_ground = ground_labels[window_points] == -1
            pred_ins[ins_ground] = 0

            pred_ins_full[window_points] = np.maximum(pred_ins_full[window_points], pred_ins)
        
    #pcd_cur = color_pcd_by_labels(merged_chunk,pred_ins_full)
    #o3d.visualization.draw_geometries([pcd_cur])
    
    colors_gen = generate_random_colors(500)
    
    # Reproject cluster labels to the original point cloud size
    cluster_labels = np.ones((len(merge_orig.points), 1)) * -1
    labels_orig = kDTree_1NN_feature_reprojection(cluster_labels, merge_orig, pred_ins_full.reshape(-1,1), merged_chunk)
    colors = np.zeros((labels_orig.shape[0],3))
    unique_labels = list(np.unique(labels_orig))
    
    for j in unique_labels:
            cur_idcs = np.where(labels_orig == j)[0]
            if j == 0 : 
                colors[cur_idcs] = np.array([0,0,0])
                
            else : 
                colors[cur_idcs] = np.array(colors_gen[unique_labels.index(j)])
                
        
    merge_orig.colors = o3d.utility.Vector3dVector(colors / 255.)
    
    return merge_orig

Here we define the dataset depending on nuscenes sequence!

In [ ]:
#DATASET_PATH = '/media/cedric/Datasets1/nuScenes_mini_v2/nuScenes'
DATASET_PATH = '/media/cedric/Datasets1/nuScenes_train'


dist_threshold = 5 #moving object filter threshold 
dataset_type = 'v1.0-trainval'

minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 
out_folder_ncuts = 'test_data/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

out_folder = 'pcd_preprocessed_nuscenes/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)

out_3duis = f'{out_folder}out_nuscenes_3duis/'
map_out_pred = f'{out_folder}out_nuscenes_3duis/maps/'

if os.path.exists(map_out_pred) == False : 
                os.makedirs(map_out_pred)




out_nuscenes_instances = f'{out_folder}out_nuscenes_instance/'
map_out_instances = f'{out_folder}out_nuscenes_instance/maps/'

alpha = 1.0
theta = 0.0
beta = 0.0
gamma = 0.0
proximity_threshold = 1.0
T = 0.05
seqs = list(range(0,85)) ##currently downloaded up to 95 range 

all_instances = []




colors = generate_random_colors_map(600,0)
for SEQUENCE_NUM in tqdm(seqs) : 
        print('current sequence',SEQUENCE_NUM)
        dataset = create_nuscenes_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True, sam_folder_name="SAM", 
                        dinov2_folder_name="Dinov2",dist_threshold=dist_threshold,dataset_type=dataset_type)
        
        ind_start = 0
        ind_end = len(dataset)  
                        
        #if os.path.exists(f'{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)
        
        
        #if os.path.exists(f'{out_folder}pcd_ground_minor' + str(SEQUENCE_NUM) + '.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)

        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}nuscenes_labels_preprocessed.npz',
                                                instance_nonground= labels['instance_nonground'],
                                                instance_ground= labels['instance_ground'],
                                                seg_ground = labels['seg_ground'],
                                                seg_nonground= labels['seg_nonground']
                                                )
        
        
        pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor.pcd')
        pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor.pcd')
        
        nuscenes_labels_orig = {}
        with np.load(f'{out_folder}nuscenes_labels_preprocessed.npz') as data :
                nuscenes_labels_orig['instance_ground'] = data['instance_ground']
                nuscenes_labels_orig['instance_nonground'] = data['instance_nonground']
                nuscenes_labels_orig['seg_nonground'] = data['seg_nonground']
                nuscenes_labels_orig['seg_ground'] = data['seg_ground']
        
                
        
        with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
                all_poses = data['all_poses']
                T_pcd = data['T_pcd']
                first_position = T_pcd[:3, 3]
        
        
        pcd_new = o3d.geometry.PointCloud()
        pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points))
        
        map_labelled = color_pcd_by_labels(pcd_new,\
                        nuscenes_labels_orig['instance_nonground'].reshape(-1,1))
        
        #o3d.visualization.draw_geometries([map_labelled])
        
        poses, positions, \
        sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)
        
        pcd_nonground_chunks, pcd_ground_chunks,\
        pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
        indices,indices_ground, center_positions, \
        center_ids, chunk_bounds, nuscenes_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                                    first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                                    overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=nuscenes_labels_orig)
                                                                    
                                                                    
        
        
        
        cams = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT"]
        cam_ids = [0]
        
        #out_dbscan = 'out_dbscan/'
        #if os.path.exists(out_dbscan) == True : 
        #        shutil.rmtree(out_dbscan)
        out_3duis_cur = out_3duis + str(SEQUENCE_NUM) + '/'
        if os.path.exists(out_3duis_cur) == True : 
                shutil.rmtree(out_3duis_cur)
        os.makedirs(out_3duis_cur)
        
        
        
        instances = np.hstack((nuscenes_labels_orig['instance_nonground'].reshape(-1,),nuscenes_labels_orig['instance_ground'].reshape(-1,)))
        
        patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
        out_data = []
        print(len(center_ids))
        for sequence in range(len(center_ids)):
                        pcd_3duis = UIS3D_clustering(pcd_nonground_chunks[sequence],pcd_ground_chunks[sequence],center_ids[sequence],
                        center_positions[sequence],
                        eps=0.4, min_samples=10)
                
                        #kitti_chunk_instance = color_pcd_by_labels(obstacle_chunk,kitti_labels['nonground']['instance'][sequence][in_idcs].reshape(-1,),
                        
                        #o3d.visualization.draw_geometries([obstacle_chunk + ground_chunk])
                        #print(kitti_chunk_instance,obstacle_chunk)
                        
                        name =  str(center_ids[sequence]).zfill(6) + '.pcd'
                        
                        o3d.io.write_point_cloud(out_3duis_cur + name, pcd_3duis, write_ascii=False, compressed=False, print_progress=False)


In [ ]:
colors = generate_random_colors_map(400,0)

def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                        
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
#out_dbscan = 'out_dbscan/'

for i in seqs: 
    print('cur seq',i)
    out_instance_cur = out_nuscenes_instances + str(i) + '/'
    out_3duis_cur = out_3duis + str(i) + '/'
    
    point_clouds = get_merge_pcds(out_3duis_cur)
    merge = merge_chunks_unite_instances(point_clouds)
    
    point_clouds_nuscenes_instances = get_merge_pcds(out_instance_cur)
    merge_nuscenes_instance = merge_unite_gt(point_clouds_nuscenes_instances)
    #o3d.visualization.draw_geometries([merge_nuscenes_instance])
    unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
    unique_colors, labels_nuscenes = np.unique(np.asarray(merge_nuscenes_instance.colors),axis=0, return_inverse=True)
        
    pred_instance = remove_semantics(labels_nuscenes,labels_ncuts)
    
    o3d.io.write_point_cloud(map_out_instances + "merge_part_nuscenes_instance" + str(i)  + ".pcd", merge_nuscenes_instance, write_ascii=False, compressed=False, print_progress=False)
    o3d.io.write_point_cloud(map_out_pred + "merge_part_nuscenes_3duis" + str(i)  + ".pcd", merge, write_ascii=False, compressed=False, print_progress=False)
    o3d.io.write_point_cloud(map_out_pred + "merge_part_nuscenes_3duis_instances_" + str(i)  + ".pcd", color_pcd_by_labels(merge,pred_instance), write_ascii=False, compressed=False, print_progress=False)
    




In [ ]:

merge_pcd_pred = o3d.geometry.PointCloud()
merge_pcd_instance = o3d.geometry.PointCloud()
merge_instances_only = o3d.geometry.PointCloud()

for i in seqs : 
	merge_pcd_pred +=  o3d.io.read_point_cloud(map_out_pred + "merge_part_nuscenes_3duis" + str(i)  + ".pcd")
	cur_gt = o3d.io.read_point_cloud(map_out_instances + "merge_part_nuscenes_instance" + str(i)  + ".pcd")
	merge_pcd_instance += cur_gt
	merge_instances_only += o3d.io.read_point_cloud(map_out_pred + "merge_part_nuscenes_3duis_instances_" + str(i)  + ".pcd")
	
unique_colors, labels_ncuts = np.unique(np.asarray(merge_pcd_pred.colors), axis=0, return_inverse=True)
#unique_colors, labels_dbscan = np.unique(np.asarray(merge_dbscan.colors), axis=0, return_inverse=True)
unique_colors, labels_nuscenes = np.unique(np.asarray(merge_pcd_instance.colors),axis=0, return_inverse=True)


#new_ncuts_labels = remove_semantics(labels_nuscenes,labels_ncuts)
unique_colors, new_ncuts_labels = np.unique(np.asarray(merge_instances_only.colors), axis=0, return_inverse=True)


metrics_ncuts = Metrics(name='dbscan')
metrics_ncuts.min_points = 50
metrics_ncuts.update_stats(new_ncuts_labels,new_ncuts_labels,labels_nuscenes)

#merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
#o3d.visualization.draw_geometries([merge_vis])
